# COMPASS Engine: Interactive Demo

### Clinical Orchestrated Multi-modal Predictive Agentic Support System

Welcome to the **COMPASS** interactive demo. This notebook demonstrates the system's reasoning capabilities using synthetic clinical data (Pseudodata).

**Objective**: To diagnose complex neuropsychiatric conditions by orchestrating multiple AI agents.

**Key Features Demonstrated**:
- **Dynamic Phenotype Targeting**: You select the diagnosis to hunt for.
- **Transparent Reasoning**: View the exact thoughts of the Orchestrator, Tools, and Predictor.
- **Multi-Agent Collaboration**: Orchestrator (Plan) -> Executor (Do) -> Integrator (Fuse) -> Predictor (Solve).

In [ ]:
import sys
import json
import ipywidgets as widgets
from IPython.display import display, Markdown, JSON
from pathlib import Path

# Ensure the system path includes the project root (Parent of multi_agent_system)
# This allows imports like 'from multi_agent_system.main import ...' to work
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from multi_agent_system.main import run_compass_pipeline
from multi_agent_system.config.settings import get_settings, LLMBackend

# Settings
PSEUDO_DATA_ROOT = Path("data/pseudo_data")

print("✅ COMPASS Engine Loaded")

## 1. Select Participant, Target & Backend

Choose a synthetic profile to analyze and select your LLM backend (OpenAI or Local).
*   **SUBJ_001**: A profile with features strongly suggestive of Major Depressive Disorder (MDD).
*   **SUBJ_002**: A 'Healthy Control' profile with minor incidental findings.

In [ ]:
# Interactive Widgets
subject_dropdown = widgets.Dropdown(
    options=[('Subject 001 (MDD Phenotype)', 'SUBJ_001_PSEUDO'), ('Subject 002 (Healthy Control)', 'SUBJ_002_PSEUDO')],
    value='SUBJ_001_PSEUDO',
    description='Subject:',
)

target_input = widgets.Text(
    value='Major Depressive Disorder',
    description='Target:',
    placeholder='e.g. Anxiety'
)

# Backend Selection
backend_dropdown = widgets.Dropdown(
    options=[('OpenAI API (GPT-5)', 'openai'), ('Local LLM (Open Source)', 'local')],
    value='openai',
    description='Backend:',
)

model_input = widgets.Text(
    value='Qwen/Qwen2.5-0.5B-Instruct',
    description='Local Model:',
    disabled=True
)

def on_backend_change(change):
    if change['new'] == 'local':
        model_input.disabled = False
    else:
        model_input.disabled = True

backend_dropdown.observe(on_backend_change, names='value')

display(subject_dropdown, target_input, backend_dropdown, model_input)

## 2. Run Main Pipeline

This step launches the **Actor-Critic Loop**.

**Watch the Output Below** for real-time logs:
1.  **Orchestrator**: "I need to check the MRI volumes and Lipidomics first..."
2.  **Executor**: Runs `PhenotypeRepresentation` tool.
3.  **Integrator (Fusion)**: Merges the tool outputs with the raw clinical notes.
4.  **Predictor**: "Based on the hippocampal atrophy and elevated CRP..."
5.  **Critic**: "Is this evidence sufficient? Yes."

In [ ]:
PARTICIPANT_DIR = PSEUDO_DATA_ROOT / subject_dropdown.value
TARGET = target_input.value

# Apply Backend Settings
settings = get_settings()
if backend_dropdown.value == 'local':
    settings.models.backend = LLMBackend.LOCAL
    settings.models.local_model_name = model_input.value
    print(f"⚙️ Configured for LOCAL inference using {model_input.value}")
else:
    settings.models.backend = LLMBackend.OPENAI
    print("⚙️ Configured for OPENAI inference")

print(f"🚀 INITIATING ANALYSIS: {subject_dropdown.value} -> {TARGET}...")

# Run Pipeline (Verbose=True for transparency)
result = run_compass_pipeline(
    participant_dir=PARTICIPANT_DIR,
    target_condition=TARGET,
    max_iterations=2,
    verbose=True,      # Show agent thoughts
    interactive_ui=False # Use CLI output for Notebook clarity
)

## 3. Transparency Inspector

Let's look under the hood at the **Execution Log**. This JSON structure reveals exactly what tools were called and what data was passed.

In [ ]:
# Load the structured Execution Log
log_path = Path(result['output_dir']) / f"execution_log_{result['participant_id']}.json"

if log_path.exists():
    with open(log_path, 'r') as f:
        logs = json.load(f)
    
    # Display Decision Trace (Simplified)
    print(f"\n🔍 DECISION TRACE for {result['participant_id']}:")
    print("-" * 60)
    
    # Iterate through logs to find key agent actions
    for entry in logs:
        if entry['type'] == 'ORCHESTRATOR':
            print(f"🔵 ORCHESTRATOR PLAN: {entry['data']['total_steps']} steps planned.")
        elif entry['type'] == 'PREDICTOR':
            print(f"🟢 PREDICTOR CONCLUSION: {entry['data']['classification']} ({entry['data']['probability']:.1%})")
        elif entry['type'] == 'CRITIC':
            print(f"🟠 CRITIC VERDICT: {entry['data']['verdict']} (Confidence: {entry['data']['confidence']})")
            
    print("-" * 60)
    # print("Full log available at:", log_path)
    print("Recent Logs:")
    print(json.dumps(logs[-3:], indent=2)) 
else:
    print("Log file not found.")

## 4. Final Clinical Report

The system generates a human-readable report. Here is the final output presented to the clinician.

In [ ]:
report_path = Path(result['output_dir']) / f"report_{result['participant_id']}.md"

if report_path.exists():
    with open(report_path, 'r') as f:
        report_content = f.read()
    display(Markdown(report_content))
else:
    print("Report not found.")